In [1]:
"""
Simple demo using kernel activation functions with convolutional networks on the MNIST dataset.
"""

# Import TensorFlow
import numpy as np
import tensorflow as tf

In [2]:
# Keras imports
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Custom imports
from kaf import Kaf
import tqdm

In [3]:
# Load Breast Cancer dataset
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

# Preprocessing is taken from here:
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [4]:
# Initialize a KAF neural network
kafnet = Sequential()
kafnet.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
kafnet.add(Kaf(20, conv=True))
kafnet.add(Conv2D(32, (3, 3)))
kafnet.add(Kaf(20, conv=True))
kafnet.add(MaxPooling2D(pool_size=(2, 2)))
kafnet.add(Flatten())
kafnet.add(Dense(100))
kafnet.add(Kaf(20))
kafnet.add(Dense(10, activation='softmax'))

In [5]:
# Use tf.data DataLoader
train_data = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32), y_train.astype(np.int64)))
test_data = tf.data.Dataset.from_tensor_slices((X_test.astype(np.float32), y_test.astype(np.int64)))

In [6]:
for xb, yb in train_data.batch(2):
    print(xb.shape)
    print(yb)
    break

(2, 28, 28, 1)
tf.Tensor([5 0], shape=(2,), dtype=int64)


In [7]:
kafnet(xb)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.09273768, 0.09322744, 0.0816758 , 0.1127829 , 0.10692486,
        0.11193405, 0.10395886, 0.103797  , 0.09738449, 0.09557692],
       [0.09273721, 0.09322833, 0.0816756 , 0.11278369, 0.10692544,
        0.11193448, 0.10395861, 0.10379672, 0.09738383, 0.09557612]],
      dtype=float32)>

In [14]:
# Optimizer
from tensorflow.keras import losses, optimizers, metrics

opt = optimizers.Adam()
loss = losses.SparseCategoricalCrossentropy()


kafnet.compile(optimizer=opt, loss=loss)

In [15]:
# Training
history = kafnet.fit(train_data.shuffle(1000).batch(32), epochs=5)

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 379s 202ms/step - loss: 0.5733
Epoch 2/5
1875/1875 [==============================] - 385s 205ms/step - loss: 0.1171
Epoch 3/5
1875/1875 [==============================] - 389s 208ms/step - loss: 0.0771
Epoch 4/5
1875/1875 [==============================] - 385s 205ms/step - loss: 0.0566
Epoch 5/5
1875/1875 [==============================] - 378s 201ms/step - loss: 0.0428


In [18]:
# Evaluation
acc = tf.metrics.Accuracy()
for xb, yb in test_data.batch(32):
    acc(yb, tf.argmax(kafnet(xb), axis=1))
print('Test accuracy after epoch {} is: '+ str(acc.result()))

Test accuracy after epoch {} is: tf.Tensor(0.9766, shape=(), dtype=float32)
